In [1]:
import os
import glob
import numpy as np
import pandas as pd

# Summary

In [2]:
dat = pd.read_excel("data/StrandSeq.xlsx")

In [3]:
# total reads

total_reads_list = []
for run, cell in dat[["Run", "Cell"]].values:
    total_reads = 0
    path = "results/mapping/filtered/%s/%s.flagstat" % (run, cell)
    if os.path.exists(path):
        for line in open(path):
            total_reads = int(line.split()[0])
            break
    total_reads_list.append(total_reads)
dat["TotalReads"] = total_reads_list

In [4]:
# uniq reads

uniq_reads_list = []
for run, cell in dat[["Run", "Cell"]].values:
    uniq_reads = 0
    path = "results/mapping/mark_parental/%s/%s.flagstat" % (run, cell)
    if os.path.exists(path):
        for line in open(path):
            if "primary mapped" in line:
                uniq_reads = int(line.split()[0])
                break
    uniq_reads_list.append(uniq_reads)
dat["UniqReads"] = uniq_reads_list

In [5]:
# background

background_list = []
for run, cell in dat[["Run", "Cell"]].values:
    background = np.nan
    path = "results/stat/background/%s/%s_summary.tsv" % (run, cell)
    if os.path.exists(path):
        tmp = pd.read_csv(path, sep="\t")
        background = tmp["Background"].values[0]
    background_list.append(background)
dat["Background"] = background_list

In [6]:
# GC content
mean_list = []
median_list = []
for run, cell in dat[["Run", "Cell"]].values:
    mean = np.nan
    median = np.nan
    path = "results/stat/gc_content/%s/%s_summary.tsv" % (run, cell)
    if os.path.exists(path):
        df = pd.read_csv(path, sep="\t")
        mean = df["GC.Mean"].values[0]
        median = df["GC.Median"].values[0]
    mean_list.append(mean)
    median_list.append(median)
dat["GC.Mean"] = mean_list
dat["GC.Median"] = median_list

In [7]:
# spikiness

values = []
for run, cell in dat[["Run", "Cell"]].values:
    v = np.nan
    path = "results/stat/spikiness/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        df = pd.read_csv(path, sep="\t")
        v = df["Spikiness"].values[0]
    values.append(v)
dat["Spikiness"] = values

In [8]:
# genomic depth

values = []
for run, cell in dat[["Run", "Cell"]].values:
    v = np.nan
    path = "results/stat/depth/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        df = pd.read_csv(path, sep="\t")
        v = df[df["Name"] == "Overall"]["Depth"].values[0]
    values.append(v)
dat["GenomicDepth"] = values

In [9]:
# genomic coverage

values = []
for run, cell in dat[["Run", "Cell"]].values:
    v = np.nan
    path = "results/stat/coverage/%s/%s.tsv" % (run, cell)
    if os.path.exists(path):
        df = pd.read_csv(path, sep="\t")
        v = df[df["Name"] == "Overall"]["Ratio"].values[0]
    values.append(v)
dat["GenomicCoverage"] = values

In [10]:
dat.to_excel("data/StrandSeq_Summary.xlsx", index=False)

# Quality control

In [11]:
d = pd.read_excel("data/StrandSeq_Summary.xlsx")

tmp = d[(d["Run"] == "20160928_PRJEB14185_CEU") & (d["UniqReads"] >= 800000) & (d["Background"] < 0.05)]
print("20160928_PRJEB14185_CEU:", len(tmp))
tmp.to_excel("data/StrandSeq_StrandSeq_QC_Pass.xlsx")

tmp = d[(d["Run"] == "20210729_PRJNA742746_HG001") & (d["UniqReads"] >= 2000000) & (d["Background"] < 0.05)]
print("20210729_PRJNA742746_HG001:", len(tmp))
tmp.to_excel("data/StrandSeq_OPStrandSeq_QC_Pass.xlsx")

20160928_PRJEB14185_CEU: 94
20210729_PRJNA742746_HG001: 74
